In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
wine = pd.read_csv('https://bit.ly/wine_csv_data')
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()
train_input, test_input, train_target, test_target = train_test_split(data, target, random_state= 42, test_size= 0.2)

# 랜덤 포레스트
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_jobs= -1, random_state= 42)
scores = cross_validate(rf, train_input, train_target, return_train_score= True, n_jobs= -1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


In [2]:
# 특성 중요도 : 랜덤 포레스트에서는 특성의 중요도가 골고루 배분되어 과대적합을 줄이고 일반화 성능을 높여준다.
rf.fit(train_input, train_target)
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


In [3]:
# OBB 점수 : 부트스트랩 샘플에 포함되지 않고 남은 샘플
rf = RandomForestClassifier(oob_score= True, n_jobs= -1, random_state= 42)
rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8934000384837406


In [4]:
# 엑스트라 트리 : 부트스트랩 샘플을 사용하지 않고 전체 훈련 세트 사용, 결정 트리에서 특성을 무작위로 분할
from sklearn.ensemble import ExtraTreesClassifier
et = ExtraTreesClassifier(n_jobs= -1, random_state= 42)
scores = cross_validate(et, train_input, train_target, return_train_score= True, n_jobs= -1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


In [5]:
# 엑스트라 트리의 특성 중요도
et.fit(train_input, train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


In [6]:
# 그레이디언트 부스팅
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(random_state= 42)
scores = cross_validate(gb, train_input, train_target, n_jobs= -1, return_train_score= True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))


0.8881086892152563 0.8720430147331015


In [7]:
# 그레이디언트 부스팅 결정 트리 개수 및 학습률 조절
gb = GradientBoostingClassifier(n_estimators= 500, learning_rate= 0.2, random_state= 42)
scores = cross_validate(gb, train_input, train_target, return_train_score= True, n_jobs= -1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


In [8]:
# 그레이디언트 부스팅의 특성 중요도
gb.fit(train_input, train_target)
print(gb.feature_importances_)

[0.15872278 0.68010884 0.16116839]


In [9]:
# 히스토그램 기반 그레이디언트 부스팅
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
hgb = HistGradientBoostingClassifier(random_state= 42)
score = cross_validate(hgb, train_input, train_target, return_train_score= True)
print(np.mean(score['train_score']), np.mean(score['test_score']))

0.9321723946453317 0.8801241948619236


In [10]:
# 히스토그램 기반 그레이디언트 부스팅의 특성 중요도 계산
from sklearn.inspection import permutation_importance
hgb.fit(train_input, train_target)
result = permutation_importance(hgb, train_input, train_target, n_repeats= 10, random_state= 42, n_jobs= -1)
print(result.importances_mean) # 훈련세트의 특성 중요도

result = permutation_importance(hgb, test_input, test_target, n_repeats= 10, random_state= 42, n_jobs= -1)
print(result.importances_mean)

[0.08876275 0.23438522 0.08027708]
[0.05969231 0.20238462 0.049     ]


In [11]:
hgb.score(test_input, test_target)

0.8723076923076923